<a href="https://colab.research.google.com/github/vitoriabtriz/chatbot_coord_ensino_ufpe/blob/main/CRIACOMP_Assistente_Virtual_com_OpenAI_Responses_e_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!pip install -q openai gradio tdqm

In [53]:
from google.colab import userdata
from openai import OpenAI
from tqdm import tqdm
import gradio as gr
import concurrent
import os

In [54]:
#cria a pasta onde o pdf vai ser armazenado
!mkdir -p /content/input_pdfs


In [55]:
from google.colab import files
uploaded = files.upload()

#move o pdf enviado para a pasta correta
import shutil
for filename in uploaded.keys():
    shutil.move(filename, f"/content/input_pdfs/{filename}")


Saving ilovepdf_merged.pdf to ilovepdf_merged.pdf


In [56]:
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
dir_pdfs = '/content/input_pdfs'
pdf_files = [os.path.join(dir_pdfs, f) for f in os.listdir(dir_pdfs)]

In [58]:
def upload_single_pdf(file_path: str, vector_store_id: str):
    file_name = os.path.basename(file_path)
    try:
        file_response = client.files.create(file=open(file_path, 'rb'), purpose="assistants")
        attach_response = client.vector_stores.files.create(
            vector_store_id=vector_store_id,
            file_id=file_response.id
        )
        return {"file": file_name, "status": "success"}
    except Exception as e:
        print(f"Error with {file_name}: {str(e)}")
        return {"file": file_name, "status": "failed", "error": str(e)}

def upload_pdf_files_to_vector_store(vector_store_id: str, pdf_files: list):
    stats = {"total_files": len(pdf_files), "successful_uploads": 0, "failed_uploads": 0, "errors": []}

    print(f"{len(pdf_files)} PDF files to process. Uploading in parallel...")

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(upload_single_pdf, file_path, vector_store_id): file_path for file_path in pdf_files}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(pdf_files)):
            result = future.result()
            if result["status"] == "success":
                stats["successful_uploads"] += 1
            else:
                stats["failed_uploads"] += 1
                stats["errors"].append(result)

    return stats

def create_vector_store(store_name: str) -> dict:
    try:
        vector_store = client.vector_stores.create(name=store_name)
        details = {
            "id": vector_store.id,
            "name": vector_store.name,
            "created_at": vector_store.created_at,
            "file_count": vector_store.file_counts.completed
        }
        print("Vector store created:", details)
        return details
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return {}

In [59]:
store_name = "coordenacao_ensino"
vector_store_details = create_vector_store(store_name)
upload_pdf_files_to_vector_store(vector_store_details["id"], pdf_files)

Vector store created: {'id': 'vs_67fee2739db88191a9e11a29881cc2a7', 'name': 'coordenacao_ensino', 'created_at': 1744757363, 'file_count': 0}
2 PDF files to process. Uploading in parallel...
Error with .ipynb_checkpoints: [Errno 21] Is a directory: '/content/input_pdfs/.ipynb_checkpoints'


100%|██████████| 2/2 [00:01<00:00,  1.56it/s]


{'total_files': 2,
 'successful_uploads': 1,
 'failed_uploads': 1,
 'errors': [{'file': '.ipynb_checkpoints',
   'status': 'failed',
   'error': "[Errno 21] Is a directory: '/content/input_pdfs/.ipynb_checkpoints'"}]}

In [60]:
import concurrent.futures

def response_output(query, history):
    def make_request():
        return client.responses.create(
            model="gpt-4o-mini",
            input=[
                {
                    "role": "system",
                    "content": "Você é um assistente virtual que responde apenas com base nos documentos fornecidos sobre a coordenação do CIn/UFPE. Caso a pergunta não tenha relação com os dados, responda: 'Desculpe, não sei informar sobre isso.'"
                },
                {"role": "user", "content": query}
            ],
            tools=[
                {
                    "type": "file_search",
                    "vector_store_ids": [vector_store_details['id']],
                }
            ]
        )

    try:
        #executa com timeout de 30 segundos
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(make_request)
            response = future.result(timeout=30)

        #verifica se veio resposta relevante de um arquivo
        if (
            hasattr(response, "output")
            and len(response.output) > 1
            and hasattr(response.output[1], "content")
            and response.output[1].content
            and response.output[1].content[0].text.strip() != ""
        ):
            return response.output[1].content[0].text.strip()
        else:
            return "Desculpe, não sei informar sobre isso."

    except concurrent.futures.TimeoutError:
        return "Desculpe, não sei informar sobre isso."
    except Exception as e:
        #você pode logar o erro se quiser: print(str(e))
        return "Desculpe, não sei informar sobre isso."


In [61]:
demo = gr.ChatInterface(
    response_output,
    type="messages"
)

demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e91113b9696c7d36e2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://e91113b9696c7d36e2.gradio.live
